In [ ]:
%matplotlib inline

In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

Translation with a Sequence to Sequence Network
*************************************************************

::

    [KEY: > input, = target, < output]

    > il est en train de peindre un tableau .
    = he is painting a picture .
    < he is painting a picture .

    > pourquoi ne pas essayer ce vin delicieux ?
    = why not try that delicious wine ?
    < why not try that delicious wine ?

    > elle n est pas poete mais romanciere .
    = she is not a poet but a novelist .
    < she not not a poet but a novelist .

    > vous etes trop maigre .
    = you re too skinny .
    < you re all alone .

... to varying degrees of success.

This is made possible by the simple but powerful idea of the `sequence
to sequence network <http://arxiv.org/abs/1409.3215>`__, in which two
recurrent neural networks work together to transform one sequence to
another. An encoder network condenses an input sequence into a vector,
and a decoder network unfolds that vector into a new sequence.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

To improve upon this model we'll use an `attention
mechanism <https://arxiv.org/abs/1409.0473>`__, which lets the decoder
learn to focus over a specific range of the input sequence.

**Recommended Reading:**

I assume you have at least installed PyTorch, know Python, and
understand Tensors:

-  http://pytorch.org/ For installation instructions
-  :doc:`/beginner/deep_learning_60min_blitz` to get started with PyTorch in general
-  :doc:`/beginner/pytorch_with_examples` for a wide and deep overview
-  :doc:`/beginner/former_torchies_tutorial` if you are former Lua Torch user


It would also be useful to know about Sequence to Sequence networks and
how they work:

-  `Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation <http://arxiv.org/abs/1406.1078>`__
-  `Sequence to Sequence Learning with Neural
   Networks <http://arxiv.org/abs/1409.3215>`__
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`__
-  `A Neural Conversational Model <http://arxiv.org/abs/1506.05869>`__

You will also find the previous tutorials on
:doc:`/intermediate/char_rnn_classification_tutorial`
and :doc:`/intermediate/char_rnn_generation_tutorial`
helpful as those concepts are very similar to the Encoder and Decoder
models, respectively.

And for more, read the papers that introduced these topics:

-  `Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation <http://arxiv.org/abs/1406.1078>`__
-  `Sequence to Sequence Learning with Neural
   Networks <http://arxiv.org/abs/1409.3215>`__
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`__
-  `A Neural Conversational Model <http://arxiv.org/abs/1506.05869>`__


**Requirements**



Loading data files
==================

The data for this project is a set of many thousands of English to
French translation pairs.

`This question on Open Data Stack
Exchange <http://opendata.stackexchange.com/questions/3888/dataset-of-sentences-translated-into-many-languages>`__
pointed me to the open translation site http://tatoeba.org/ which has
downloads available at http://tatoeba.org/eng/downloads - and better
yet, someone did the extra work of splitting language pairs into
individual text files here: http://www.manythings.org/anki/

The English to French pairs are too big to include in the repo, so
download to ``data/eng-fra.txt`` before continuing. The file is a tab
separated list of translation pairs:

::

    I am cold.    J'ai froid.

.. Note::
   Download the data from
   `here <https://download.pytorch.org/tutorial/data.zip>`_
   and extract it to the current directory.



In [ ]:
!wget http://www.manythings.org/anki/fra-eng.zip
!unzip -o fra-eng.zip
!mkdir data
!mv fra.txt data/eng-fra.txt

--2025-04-08 07:49:45--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.57M  4.36MB/s    in 1.7s    

2025-04-08 07:49:47 (4.36 MB/s) - ‘fra-eng.zip’ saved [7943074/7943074]

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.




In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data file we will split the file into lines, and then split
lines into pairs. The files are all English → Other Language, so if we
want to translate from Other Language → English I added the ``reverse``
flag to reverse the pairs.




In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Since there are a *lot* of example sentences and we want to train
something quickly, we'll trim the data set to only relatively short and
simple sentences. Here the maximum length is 10 words (that includes
ending punctuation) and we're filtering to sentences that translate to
the form "I am" or "He is" etc. (accounting for apostrophes replaced
earlier).




In [ ]:
MAX_LENGTH = 15

eng_prefixes = (
    "i am", "i m",
    "he is", "he s",
    "she is", "she s",
    "you are", "you re",
    "we are", "we re",
    "they are", "they re"
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
#print(random.choice(pairs))

Reading lines...
Read 232736 sentence pairs
Trimmed to 22907 sentence pairs
Counting words...
Counted words:
fra 7019
eng 4638


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = [i[0] for i in pairs]
y = [i[1] for i in pairs]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
train_pairs = list(zip(X_train,y_train))
test_pairs = list(zip(X_test,y_test))

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <http://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence "Je ne suis pas le chat noir" → "I am not the
black cat". Most of the words in the input sentence have a direct
translation in the output sentence, but are in slightly different
orders, e.g. "chat noir" and "black cat". Because of the "ne/pas"
construction there is also one more word in the input sentence. It would
be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.






In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder (Your assignment)
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




Simple Decoder
^^^^^^^^^^^^^^

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).


In [ ]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):

        # Your code here #
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`__.</p></div>

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`__.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [ ]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [ ]:
def trainIters(encoder, decoder, epochs, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    criterion = nn.NLLLoss()

    iter = 1
    n_iters = len(train_pairs) * epochs

    for epoch in range(epochs):
        print("Epoch: %d/%d" % (epoch, epochs))
        for training_pair in train_pairs:
            training_pair = tensorsFromPair(training_pair)

            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                            iter, iter / n_iters * 100, print_loss_avg))

            iter +=1

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
from torchmetrics.text.rouge import ROUGEScore
from tqdm import tqdm
import numpy as np

def inference(encoder, decoder, testing_pairs):
    input = []
    gt = []
    predict = []

    from tqdm import tqdm
    for i in tqdm(range(len(testing_pairs))):
        pair = testing_pairs[i]
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)

        input.append(pair[0])
        gt.append(pair[1])
        predict.append(output_sentence)

    return input,gt,predict


def eval(gt, predict):
  rouge = ROUGEScore()
  metric_score = rouge(predict, gt)
  print("=== Evaluation score - Rouge score ===")
  print("Rouge1 fmeasure:\t",metric_score["rouge1_fmeasure"].item())
  print("Rouge1 precision:\t",metric_score["rouge1_precision"].item())
  print("Rouge1 recall:  \t",metric_score["rouge1_recall"].item())
  print("Rouge2 fmeasure:\t",metric_score["rouge2_fmeasure"].item())
  print("Rouge2 precision:\t",metric_score["rouge2_precision"].item())
  print("Rouge2 recall:  \t",metric_score["rouge2_recall"].item())
  print("=====================================")

Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = Decoder(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 5, print_every=5000)

Epoch: 0/5
1m 21s (- 26m 34s) (5000 4%) 3.3438
2m 43s (- 25m 19s) (10000 9%) 2.8515
4m 6s (- 24m 8s) (15000 14%) 2.5794
5m 30s (- 22m 52s) (20000 19%) 2.3968
Epoch: 1/5
6m 53s (- 21m 32s) (25000 24%) 2.2277
8m 16s (- 20m 10s) (30000 29%) 2.0780
9m 40s (- 18m 50s) (35000 33%) 1.9383
11m 4s (- 17m 28s) (40000 38%) 1.8506
Epoch: 2/5
12m 27s (- 16m 4s) (45000 43%) 1.7583
13m 51s (- 14m 43s) (50000 48%) 1.6594
15m 15s (- 13m 20s) (55000 53%) 1.5802
16m 38s (- 11m 56s) (60000 58%) 1.5188
Epoch: 3/5
18m 1s (- 10m 33s) (65000 63%) 1.4454
19m 25s (- 9m 10s) (70000 67%) 1.3762
20m 48s (- 7m 47s) (75000 72%) 1.3309
22m 13s (- 6m 24s) (80000 77%) 1.2636
Epoch: 4/5
23m 37s (- 5m 1s) (85000 82%) 1.2085
25m 2s (- 3m 38s) (90000 87%) 1.1714
26m 26s (- 2m 14s) (95000 92%) 1.1190
27m 50s (- 0m 51s) (100000 97%) 1.0672


In [ ]:
evaluateRandomly(encoder1, decoder1)

> je suis vraiment desole d etre rentre a la maison aussi tard .
= i m very sorry i came home so late .
< i m very sorry that i came so late . <EOS>

> tu ne reponds pas a la question .
= you re not answering the question .
< you re not answering the question . <EOS>

> elle a l esprit ouvert .
= she s open minded .
< she is open minded . <EOS>

> je suis heureux de l entendre .
= i m glad to hear that .
< i m glad to hear that . <EOS>

> je vais resoudre ce probleme tout seul .
= i am going to work out the problem by myself .
< i m going to work this problem by problem . <EOS>

> je suis quelque peu decu .
= i m a little disappointed .
< i m a little disappointed . <EOS>

> c est une jeune femme tres intelligente .
= she is a very intelligent young lady .
< she is a very young man young . <EOS>

> tu es fort comprehensive .
= you re very understanding .
< you re very understanding . <EOS>

> vous vous comportez tous les deux comme des enfants .
= you re both acting like children .
< y

In [ ]:
# input,gt,predict = inference(encoder1, decoder1, train_pairs)
# eval(gt, predict)

In [ ]:
input,gt,predict = inference(encoder1, decoder1, test_pairs)
eval(gt, predict)

100%|██████████| 2291/2291 [00:13<00:00, 170.97it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.6222008466720581
Rouge1 precision:	 0.5851839780807495
Rouge1 recall:  	 0.6731113195419312
Rouge2 fmeasure:	 0.445445716381073
Rouge2 precision:	 0.4114634692668915
Rouge2 recall:  	 0.494349867105484


In [ ]:
# ------------------------------
# Cell 1: Results dictionary and ROUGE helper
# ------------------------------
results = {}

def get_rouge_scores(ground_truth, predictions):
    from torchmetrics.text.rouge import ROUGEScore
    rouge = ROUGEScore()
    metric_score = rouge(predictions, ground_truth)
    return {
      "rouge1_fmeasure": metric_score["rouge1_fmeasure"].item(),
      "rouge1_precision": metric_score["rouge1_precision"].item(),
      "rouge1_recall": metric_score["rouge1_recall"].item(),
      "rouge2_fmeasure": metric_score["rouge2_fmeasure"].item(),
      "rouge2_precision": metric_score["rouge2_precision"].item(),
      "rouge2_recall": metric_score["rouge2_recall"].item()
    }


In [ ]:
# Run inference on the test set with your base GRU model (encoder1/decoder1)
input_base, gt_base, pred_base = inference(encoder1, decoder1, test_pairs)
base_scores = get_rouge_scores(gt_base, pred_base)
results["Base"] = base_scores

print("Base model ROUGE scores:")
print(base_scores)


100%|██████████| 2291/2291 [00:13<00:00, 174.02it/s]


Base model ROUGE scores:
{'rouge1_fmeasure': 0.6222008466720581, 'rouge1_precision': 0.5851839780807495, 'rouge1_recall': 0.6731113195419312, 'rouge2_fmeasure': 0.445445716381073, 'rouge2_precision': 0.4114634692668915, 'rouge2_recall': 0.494349867105484}


In [ ]:
# ------------------------------
# Cell 2: Task 2 – LSTM Encoder and Decoder
# ------------------------------
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# LSTM-based Encoder: uses nn.LSTM instead of GRU.
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

# LSTM-based Decoder: uses nn.LSTM instead of GRU.
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        # input: tensor of shape [1, 1]
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

# New training function adapted for LSTM (hidden is a tuple).
def trainLSTM(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden  # (hidden, cell)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

# A minimal training loop function for the LSTM experiment.
def trainIters_LSTM(encoder, decoder, n_epochs=5, learning_rate=0.01, print_every=5000):
    start = time.time()
    print_loss_total = 0
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    n_iters = len(train_pairs) * n_epochs
    iter_count = 1
    for epoch in range(n_epochs):
        print("LSTM Epoch: %d/%d" % (epoch+1, n_epochs))
        for pair in train_pairs:
            training_pair = tensorsFromPair(pair)
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]
            loss = trainLSTM(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            if iter_count % print_every == 0:
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter_count / n_iters),
                                             iter_count, iter_count / n_iters * 100, print_loss_total/print_every))
                print_loss_total = 0
            iter_count += 1

# Instantiate the new LSTM models.
encoder_lstm = EncoderLSTM(input_lang.n_words, hidden_size).to(device)
decoder_lstm = DecoderLSTM(hidden_size, output_lang.n_words).to(device)

# Train the LSTM-based seq2seq (adjust n_epochs as desired).
trainIters_LSTM(encoder_lstm, decoder_lstm, n_epochs=5, print_every=5000)

# Evaluate on test set.
def evaluateLSTM(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        encoder_hidden = encoder.initHidden()

        input_length = input_tensor.size(0)
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden

        decoded_words = []
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        return decoded_words

def inference_LSTM(encoder, decoder, testing_pairs):
    inputs, gts, predicts = [], [], []
    for pair in testing_pairs:
        output_words = evaluateLSTM(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        inputs.append(pair[0])
        gts.append(pair[1])
        predicts.append(output_sentence)
    return inputs, gts, predicts

inp_lstm, gt_lstm, pred_lstm = inference_LSTM(encoder_lstm, decoder_lstm, test_pairs)
lstm_scores = get_rouge_scores(gt_lstm, pred_lstm)
results["LSTM"] = lstm_scores

print("LSTM variant ROUGE scores:")
print(lstm_scores)


LSTM Epoch: 1/5
1m 29s (- 29m 21s) (5000 4%) 3.4458
3m 2s (- 28m 20s) (10000 9%) 3.0445
4m 35s (- 26m 59s) (15000 14%) 2.8146
6m 8s (- 25m 32s) (20000 19%) 2.6441
LSTM Epoch: 2/5
7m 42s (- 24m 3s) (25000 24%) 2.4786
9m 15s (- 22m 32s) (30000 29%) 2.3297
10m 54s (- 21m 12s) (35000 33%) 2.2141
12m 28s (- 19m 39s) (40000 38%) 2.1236
LSTM Epoch: 3/5
14m 1s (- 18m 6s) (45000 43%) 2.0332
15m 35s (- 16m 33s) (50000 48%) 1.9298
17m 9s (- 14m 59s) (55000 53%) 1.8350
18m 42s (- 13m 25s) (60000 58%) 1.7925
LSTM Epoch: 4/5
20m 17s (- 11m 53s) (65000 63%) 1.7108
21m 50s (- 10m 19s) (70000 67%) 1.6532
23m 23s (- 8m 45s) (75000 72%) 1.5848
24m 56s (- 7m 11s) (80000 77%) 1.5436
LSTM Epoch: 5/5
26m 30s (- 5m 38s) (85000 82%) 1.4830
28m 4s (- 4m 4s) (90000 87%) 1.4143
29m 37s (- 2m 31s) (95000 92%) 1.3726
31m 9s (- 0m 57s) (100000 97%) 1.3385
LSTM variant ROUGE scores:
{'rouge1_fmeasure': 0.6070468425750732, 'rouge1_precision': 0.5753817558288574, 'rouge1_recall': 0.6506457328796387, 'rouge2_fmeasure': 

In [ ]:
# ------------------------------
# Cell 3: Task 3 – BiLSTM Encoder variant (decoder remains as original GRU)
# ------------------------------
class EncoderBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderBiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        # bidirectional LSTM with batch_first=True
        self.lstm = nn.LSTM(hidden_size, hidden_size, bidirectional=True, batch_first=True)
        # linear layer to project concatenated outputs (hidden_size*2) to hidden_size
        self.fc = nn.Linear(hidden_size * 2, hidden_size)

    def forward(self, input):
        # input: tensor of shape (seq_len, 1) -> transpose to (1, seq_len)
        input = input.transpose(0, 1)  # shape: (1, seq_len)
        embedded = self.embedding(input)  # shape: (1, seq_len, hidden_size)
        outputs, (hn, cn) = self.lstm(embedded)  # outputs shape: (1, seq_len, hidden_size*2)
        # take the mean over the time dimension
        mean_output = outputs.mean(dim=1)  # shape: (1, hidden_size*2)
        sentence_representation = self.fc(mean_output)  # shape: (1, hidden_size)
        return sentence_representation

# For this experiment we reuse the original GRU-based Decoder.
# New training function for BiLSTM encoder variant.
def trainBiLSTM(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Encoder processes the whole sentence at once.
    encoder_sentence_rep = encoder(input_tensor)  # shape (1, hidden_size)
    # Prepare decoder initial hidden state (GRU expects shape: (1, 1, hidden_size))
    decoder_hidden = encoder_sentence_rep.unsqueeze(0)

    loss = 0
    decoder_input = torch.tensor([[SOS_token]], device=device)
    target_length = target_tensor.size(0)
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        loss += criterion(decoder_output, target_tensor[di])
        # Teacher forcing: always use the target here.
        decoder_input = target_tensor[di]
        if decoder_input.item() == EOS_token:
            break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

def trainIters_BiLSTM(encoder, decoder, n_epochs=5, learning_rate=0.01, print_every=5000):
    start = time.time()
    print_loss_total = 0
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    n_iters = len(train_pairs) * n_epochs
    iter_count = 1
    for epoch in range(n_epochs):
        print("BiLSTM Epoch: %d/%d" % (epoch+1, n_epochs))
        for pair in train_pairs:
            training_pair = tensorsFromPair(pair)
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]
            loss = trainBiLSTM(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            if iter_count % print_every == 0:
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter_count / n_iters),
                                             iter_count, iter_count / n_iters * 100, print_loss_total/print_every))
                print_loss_total = 0
            iter_count += 1

def evaluateBiLSTM(encoder, decoder, sentence):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        encoder_sentence_rep = encoder(input_tensor)
        decoder_hidden = encoder_sentence_rep.unsqueeze(0)
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoded_words = []
        for di in range(MAX_LENGTH):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        return decoded_words

def inference_BiLSTM(encoder, decoder, testing_pairs):
    inputs, gts, predicts = [], [], []
    for pair in testing_pairs:
        output_words = evaluateBiLSTM(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        inputs.append(pair[0])
        gts.append(pair[1])
        predicts.append(output_sentence)
    return inputs, gts, predicts

# Instantiate the new bidirectional encoder.
encoder_bilstm = EncoderBiLSTM(input_lang.n_words, hidden_size).to(device)
# Use the original GRU-based decoder.
decoder_bilstm = Decoder(hidden_size, output_lang.n_words).to(device)

# Train the BiLSTM variant.
trainIters_BiLSTM(encoder_bilstm, decoder_bilstm, n_epochs=5, print_every=5000)

# Evaluate on test set.
inp_bilstm, gt_bilstm, pred_bilstm = inference_BiLSTM(encoder_bilstm, decoder_bilstm, test_pairs)
bilstm_scores = get_rouge_scores(gt_bilstm, pred_bilstm)
results["BiLSTM"] = bilstm_scores

print("BiLSTM Encoder ROUGE scores:")
print(bilstm_scores)


BiLSTM Epoch: 1/5
1m 3s (- 20m 55s) (5000 4%) 3.1956
2m 8s (- 19m 53s) (10000 9%) 2.5564
3m 11s (- 18m 43s) (15000 14%) 2.3265
4m 14s (- 17m 39s) (20000 19%) 2.1565
BiLSTM Epoch: 2/5
5m 18s (- 16m 34s) (25000 24%) 1.9662
6m 21s (- 15m 30s) (30000 29%) 1.8021
7m 28s (- 14m 31s) (35000 33%) 1.6809
8m 32s (- 13m 28s) (40000 38%) 1.5976
BiLSTM Epoch: 3/5
9m 37s (- 12m 25s) (45000 43%) 1.4839
10m 42s (- 11m 21s) (50000 48%) 1.3677
11m 47s (- 10m 18s) (55000 53%) 1.2714
12m 53s (- 9m 15s) (60000 58%) 1.2238
BiLSTM Epoch: 4/5
13m 59s (- 8m 11s) (65000 63%) 1.1322
15m 4s (- 7m 7s) (70000 67%) 1.0567
16m 9s (- 6m 2s) (75000 72%) 0.9774
17m 16s (- 4m 58s) (80000 77%) 0.9306
BiLSTM Epoch: 5/5
18m 21s (- 3m 54s) (85000 82%) 0.8680
19m 25s (- 2m 49s) (90000 87%) 0.8085
20m 30s (- 1m 44s) (95000 92%) 0.7507
21m 35s (- 0m 39s) (100000 97%) 0.7069
BiLSTM Encoder ROUGE scores:
{'rouge1_fmeasure': 0.5892815589904785, 'rouge1_precision': 0.5438493490219116, 'rouge1_recall': 0.6541732549667358, 'rouge2_fm

In [ ]:
# ------------------------------
# Cell 4: Task 4 – Decoder with Attention Mechanism
# ------------------------------
class AttnDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(output_size, hidden_size)
        # attention layer: input is concatenation of decoder hidden and embedded input.
        self.attn = nn.Linear(hidden_size * 2, max_length)
        self.attn_combine = nn.Linear(hidden_size * 2, hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encoder_outputs):
        # input: [1, 1]
        embedded = self.embedding(input).view(1, 1, -1)  # shape: [1, 1, hidden_size]
        embedded = self.dropout(embedded)
        # Concatenate hidden state (take hidden[0]) with embedded input.
        attn_input = torch.cat((embedded[0], hidden[0]), 1)  # shape: [1, 2*hidden_size]
        attn_weights = F.softmax(self.attn(attn_input), dim=1)  # shape: [1, max_length]
        # Compute context vector as weighted sum of encoder outputs.
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))  # shape: [1, 1, hidden_size]

        output = torch.cat((embedded[0], attn_applied[0]), 1)  # shape: [1, 2*hidden_size]
        output = self.attn_combine(output).unsqueeze(0)  # shape: [1, 1, hidden_size]
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden, attn_weights

# Training function for the attention model.
def trainAttention(input_tensor, target_tensor, encoder, attn_decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, attn_weights = attn_decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # teacher forcing
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, attn_weights = attn_decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

def trainIters_Attn(encoder, attn_decoder, n_epochs=5, learning_rate=0.01, print_every=5000):
    start = time.time()
    print_loss_total = 0
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(attn_decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    n_iters = len(train_pairs) * n_epochs
    iter_count = 1
    for epoch in range(n_epochs):
        print("Attention Epoch: %d/%d" % (epoch+1, n_epochs))
        for pair in train_pairs:
            training_pair = tensorsFromPair(pair)
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]
            loss = trainAttention(input_tensor, target_tensor, encoder, attn_decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            if iter_count % print_every == 0:
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter_count / n_iters),
                                             iter_count, iter_count / n_iters * 100, print_loss_total/print_every))
                print_loss_total = 0
            iter_count += 1

def evaluateAttention(encoder, attn_decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        encoder_hidden = encoder.initHidden()
        input_length = input_tensor.size(0)
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden

        decoded_words = []
        for di in range(max_length):
            decoder_output, decoder_hidden, attn_weights = attn_decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        return decoded_words

def inference_Attn(encoder, attn_decoder, testing_pairs):
    inputs, gts, predicts = [], [], []
    for pair in testing_pairs:
        output_words = evaluateAttention(encoder, attn_decoder, pair[0])
        output_sentence = ' '.join(output_words)
        inputs.append(pair[0])
        gts.append(pair[1])
        predicts.append(output_sentence)
    return inputs, gts, predicts

# Instantiate: use the original GRU-based encoder and new attention decoder.
attn_decoder = AttnDecoder(hidden_size, output_lang.n_words, dropout_p=0.1, max_length=MAX_LENGTH).to(device)
# Reuse your original encoder1 (GRU-based)
# Train the attention variant.
trainIters_Attn(encoder1, attn_decoder, n_epochs=5, print_every=5000)

# Evaluate on test set.
inp_attn, gt_attn, pred_attn = inference_Attn(encoder1, attn_decoder, test_pairs)
attn_scores = get_rouge_scores(gt_attn, pred_attn)
results["Attention"] = attn_scores

print("Attention variant ROUGE scores:")
print(attn_scores)


Attention Epoch: 1/5
2m 6s (- 41m 24s) (5000 4%) 2.9518
4m 15s (- 39m 35s) (10000 9%) 2.3046
6m 24s (- 37m 40s) (15000 14%) 2.0427
8m 33s (- 35m 33s) (20000 19%) 1.8946
Attention Epoch: 2/5
10m 45s (- 33m 34s) (25000 24%) 1.7569
12m 55s (- 31m 28s) (30000 29%) 1.6084
15m 5s (- 29m 20s) (35000 33%) 1.4981
17m 14s (- 27m 10s) (40000 38%) 1.4520
Attention Epoch: 3/5
19m 24s (- 25m 2s) (45000 43%) 1.3908
21m 36s (- 22m 55s) (50000 48%) 1.3149
23m 43s (- 20m 44s) (55000 53%) 1.2366
25m 49s (- 18m 32s) (60000 58%) 1.2170
Attention Epoch: 4/5
27m 55s (- 16m 21s) (65000 63%) 1.1827
29m 59s (- 14m 10s) (70000 67%) 1.1443
32m 6s (- 12m 1s) (75000 72%) 1.0899
34m 16s (- 9m 53s) (80000 77%) 1.0702
Attention Epoch: 5/5
36m 26s (- 7m 45s) (85000 82%) 1.0584
38m 34s (- 5m 36s) (90000 87%) 1.0309
40m 43s (- 3m 27s) (95000 92%) 0.9972
42m 53s (- 1m 19s) (100000 97%) 0.9582
Attention variant ROUGE scores:
{'rouge1_fmeasure': 0.6264095306396484, 'rouge1_precision': 0.5896250009536743, 'rouge1_recall': 0.

In [ ]:
# ------------------------------
# Cell 5: Task 5 – Transformer Encoder variant
# ------------------------------
import math

# Positional Encoding module.
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # Even indices
        pe[:, 0::2] = torch.sin(position * div_term)
        # Odd indices
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)  # shape: [max_len, 1, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: shape [seq_len, d_model]
        x = x + self.pe[:x.size(0), 0]
        return self.dropout(x)

# Transformer-based Encoder.
class EncoderTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, nhead=8, num_layers=2, max_length=MAX_LENGTH):
        super(EncoderTransformer, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.pos_encoder = PositionalEncoding(hidden_size, dropout=0.1, max_len=max_length)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def forward(self, input_tensor):
        # input_tensor: shape [seq_len, 1]
        # Squeeze the batch dim and get shape [seq_len, hidden_size]
        embedded = self.embedding(input_tensor).squeeze(1)
        # Apply positional encoding.
        encoded = self.pos_encoder(embedded)
        # Transformer expects shape [seq_len, batch_size, feature], so add batch dim:
        transformer_input = encoded.unsqueeze(1)
        transformer_output = self.transformer_encoder(transformer_input)
        # Obtain a sentence representation by taking the mean over the sequence dim.
        sentence_representation = transformer_output.mean(dim=0)  # shape: [1, hidden_size]
        return sentence_representation

# Training function for transformer-based encoder.
def trainTransformer(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # For Transformer encoder, feed the whole sentence at once.
    encoder_sentence_rep = encoder(input_tensor)  # shape: [1, hidden_size]
    # Set decoder initial hidden state (GRU expects shape: [1, 1, hidden_size])
    decoder_hidden = encoder_sentence_rep.unsqueeze(0)

    loss = 0
    decoder_input = torch.tensor([[SOS_token]], device=device)
    target_length = target_tensor.size(0)
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        loss += criterion(decoder_output, target_tensor[di])
        decoder_input = target_tensor[di]  # teacher forcing
        if decoder_input.item() == EOS_token:
            break
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

def trainIters_Transformer(encoder, decoder, n_epochs=5, learning_rate=0.01, print_every=5000):
    start = time.time()
    print_loss_total = 0
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    n_iters = len(train_pairs) * n_epochs
    iter_count = 1
    for epoch in range(n_epochs):
        print("Transformer Encoder Epoch: %d/%d" % (epoch+1, n_epochs))
        for pair in train_pairs:
            training_pair = tensorsFromPair(pair)
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]
            loss = trainTransformer(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            if iter_count % print_every == 0:
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter_count / n_iters),
                                             iter_count, iter_count / n_iters * 100, print_loss_total/print_every))
                print_loss_total = 0
            iter_count += 1

def evaluateTransformer(encoder, decoder, sentence):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        encoder_sentence_rep = encoder(input_tensor)  # shape: [1, hidden_size]
        decoder_hidden = encoder_sentence_rep.unsqueeze(0)
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoded_words = []
        for di in range(MAX_LENGTH):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        return decoded_words

def inference_Transformer(encoder, decoder, testing_pairs):
    inputs, gts, predicts = [], [], []
    for pair in testing_pairs:
        output_words = evaluateTransformer(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        inputs.append(pair[0])
        gts.append(pair[1])
        predicts.append(output_sentence)
    return inputs, gts, predicts

# Instantiate the Transformer encoder.
encoder_transformer = EncoderTransformer(input_lang.n_words, hidden_size, nhead=8, num_layers=2, max_length=MAX_LENGTH).to(device)
# Reuse the original GRU-based decoder.
decoder_transformer = Decoder(hidden_size, output_lang.n_words).to(device)

# Train the Transformer variant.
trainIters_Transformer(encoder_transformer, decoder_transformer, n_epochs=5, print_every=5000)

# Evaluate on test set.
inp_trans, gt_trans, pred_trans = inference_Transformer(encoder_transformer, decoder_transformer, test_pairs)
transformer_scores = get_rouge_scores(gt_trans, pred_trans)
results["Transformer"] = transformer_scores

print("Transformer Encoder variant ROUGE scores:")
print(transformer_scores)


Transformer Encoder Epoch: 1/5


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


1m 20s (- 26m 15s) (5000 4%) 3.0420
2m 38s (- 24m 39s) (10000 9%) 2.4781
3m 57s (- 23m 14s) (15000 14%) 2.2817
5m 14s (- 21m 46s) (20000 19%) 2.1399
Transformer Encoder Epoch: 2/5
6m 33s (- 20m 29s) (25000 24%) 1.9616
7m 51s (- 19m 8s) (30000 29%) 1.8330
9m 9s (- 17m 48s) (35000 33%) 1.7447
10m 27s (- 16m 29s) (40000 38%) 1.6934
Transformer Encoder Epoch: 3/5
11m 46s (- 15m 12s) (45000 43%) 1.6099
13m 4s (- 13m 52s) (50000 48%) 1.5208
14m 23s (- 12m 34s) (55000 53%) 1.4487
15m 41s (- 11m 15s) (60000 58%) 1.4384
Transformer Encoder Epoch: 4/5
17m 1s (- 9m 58s) (65000 63%) 1.3707
18m 20s (- 8m 40s) (70000 67%) 1.3194
19m 38s (- 7m 21s) (75000 72%) 1.2612
20m 55s (- 6m 2s) (80000 77%) 1.2373
Transformer Encoder Epoch: 5/5
22m 14s (- 4m 43s) (85000 82%) 1.1967
23m 34s (- 3m 25s) (90000 87%) 1.1481
24m 52s (- 2m 6s) (95000 92%) 1.0952
26m 11s (- 0m 48s) (100000 97%) 1.0625
Transformer Encoder variant ROUGE scores:
{'rouge1_fmeasure': 0.5120818018913269, 'rouge1_precision': 0.475886553525924

In [ ]:
# ------------------------------
# Cell 6: Display summary of ROUGE scores for all variants
# ------------------------------
print("Final ROUGE scores summary:")
for key, val in results.items():
    print(f"{key} model:")
    for metric, score in val.items():
        print(f"  {metric}: {score:.4f}")
    print("")


Final ROUGE scores summary:
Base model:
  rouge1_fmeasure: 0.6222
  rouge1_precision: 0.5852
  rouge1_recall: 0.6731
  rouge2_fmeasure: 0.4454
  rouge2_precision: 0.4115
  rouge2_recall: 0.4943

LSTM model:
  rouge1_fmeasure: 0.6070
  rouge1_precision: 0.5754
  rouge1_recall: 0.6506
  rouge2_fmeasure: 0.4212
  rouge2_precision: 0.3925
  rouge2_recall: 0.4625

BiLSTM model:
  rouge1_fmeasure: 0.5893
  rouge1_precision: 0.5438
  rouge1_recall: 0.6542
  rouge2_fmeasure: 0.4180
  rouge2_precision: 0.3795
  rouge2_recall: 0.4754

Attention model:
  rouge1_fmeasure: 0.6264
  rouge1_precision: 0.5896
  rouge1_recall: 0.6771
  rouge2_fmeasure: 0.4448
  rouge2_precision: 0.4108
  rouge2_recall: 0.4937

Transformer model:
  rouge1_fmeasure: 0.5121
  rouge1_precision: 0.4759
  rouge1_recall: 0.5678
  rouge2_fmeasure: 0.3373
  rouge2_precision: 0.3081
  rouge2_recall: 0.3847

